In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd /gdrive/My Drive/AN2DL/Homework1

/gdrive/My Drive/AN2DL/Homework1


### Import libraries

In [3]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.9.2


### Set seed for reproducibility

In [4]:
# Random seed for reproducibility
seed = 41

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Suppress warnings

In [5]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [6]:
# Download the data
dataset = '/gdrive/My Drive/AN2DL/Homework1/training_dataset_homework1'
training = '/gdrive/My Drive/AN2DL/Homework1/training_dataset_homework1/training'
validation = '/gdrive/My Drive/AN2DL/Homework1/training_dataset_homework1/validation'
test = '/gdrive/My Drive/AN2DL/Homework1/training_dataset_homework1/test'

In [7]:
labels = {
    0:'species1',
    1:'species2',
    2:'species3',
    3:'species4',
    4:'species5',
    5:'species6',
    6:'species7',
    7:'species8'
}

### Process data

In [26]:
# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
test_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=validation,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
test_gen = train_data_gen.flow_from_directory(directory=test,
                                              target_size=(96,96),
                                              color_mode='rgb',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=32,
                                              shuffle=False,
                                              seed=seed
                                              )

Found 2671 images belonging to 8 classes.
Found 336 images belonging to 8 classes.
Found 535 images belonging to 8 classes.


In [27]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=30,
                                        horizontal_flip= True,
                                        vertical_flip= True,
                                        brightness_range=(0.6,1.4),
                                        zoom_range=0.6,
                                        fill_mode='nearest',
                                        rescale=1./255) # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       seed=seed)

Found 2671 images belonging to 8 classes.


### Models metadata

In [28]:
input_shape = (96, 96, 3)
batch_size = 32
epochs = 200

### Standard model

### Transfer learning

In [29]:
# Download and plot the VGG16 model
supernet = tfk.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)
supernet.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 47, 47, 32)   864         ['input_3[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 47, 47, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                                  
 block1_conv1_act (Activation)  (None, 47, 47, 32)   0           ['block1_conv1_bn[0][0]'] 

In [30]:
# Use the supernet as feature extractor
supernet.trainable = False

inputs = tfk.Input(shape=(96,96,3))
x = supernet(inputs)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    256, 
    activation='relu',
    kernel_initializer = tfk.initializers.HeUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    8,
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
tl_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 Flattening (Flatten)        (None, 18432)             0         
                                                                 
 dropout_2 (Dropout)         (None, 18432)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               4718848   
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 8)                 2056  

In [31]:
# Train the model
tl_history = tl_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
84/84 [==============================] - 18s 183ms/step - loss: 2.3200 - accuracy: 0.2587 - val_loss: 1.8619 - val_accuracy: 0.2887
Epoch 2/200
84/84 [==============================] - 13s 157ms/step - loss: 1.7314 - accuracy: 0.3523 - val_loss: 1.6669 - val_accuracy: 0.4137
Epoch 3/200
84/84 [==============================] - 14s 172ms/step - loss: 1.6366 - accuracy: 0.4017 - val_loss: 1.5982 - val_accuracy: 0.4435
Epoch 4/200
84/84 [==============================] - 13s 156ms/step - loss: 1.5956 - accuracy: 0.4028 - val_loss: 1.6105 - val_accuracy: 0.4048
Epoch 5/200
84/84 [==============================] - 13s 154ms/step - loss: 1.5709 - accuracy: 0.4021 - val_loss: 1.5255 - val_accuracy: 0.4345
Epoch 6/200
84/84 [==============================] - 13s 155ms/step - loss: 1.5424 - accuracy: 0.4189 - val_loss: 1.5258 - val_accuracy: 0.4673
Epoch 7/200
84/84 [==============================] - 14s 166ms/step - loss: 1.5288 - accuracy: 0.4287 - val_loss: 1.4824 - val_accuracy:

In [32]:
# Save the best model

tl_model.save('TransferLearningModel')
del tl_model

In [ ]:
model_aug = tfk.models.load_model("TransferLearningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

17/17 [==============================] - 18s 294ms/step - loss: 1.2766 - accuracy: 0.6748

Test metrics with VGG
{'loss': 1.2766281366348267, 'accuracy': 0.6747663617134094}


### Fine tuning

In [48]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel')
ft_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 Flattening (Flatten)        (None, 18432)             0         
                                                                 
 dropout_2 (Dropout)         (None, 18432)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               4718848   
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 8)                 2056  

In [49]:
# Set all VGG layers to True
ft_model.get_layer('xception').trainable = True
for i, layer in enumerate(ft_model.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)

0 input_3 True
1 block1_conv1 True
2 block1_conv1_bn True
3 block1_conv1_act True
4 block1_conv2 True
5 block1_conv2_bn True
6 block1_conv2_act True
7 block2_sepconv1 True
8 block2_sepconv1_bn True
9 block2_sepconv2_act True
10 block2_sepconv2 True
11 block2_sepconv2_bn True
12 conv2d_4 True
13 block2_pool True
14 batch_normalization_4 True
15 add_12 True
16 block3_sepconv1_act True
17 block3_sepconv1 True
18 block3_sepconv1_bn True
19 block3_sepconv2_act True
20 block3_sepconv2 True
21 block3_sepconv2_bn True
22 conv2d_5 True
23 block3_pool True
24 batch_normalization_5 True
25 add_13 True
26 block4_sepconv1_act True
27 block4_sepconv1 True
28 block4_sepconv1_bn True
29 block4_sepconv2_act True
30 block4_sepconv2 True
31 block4_sepconv2_bn True
32 conv2d_6 True
33 block4_pool True
34 batch_normalization_6 True
35 add_14 True
36 block5_sepconv1_act True
37 block5_sepconv1 True
38 block5_sepconv1_bn True
39 block5_sepconv2_act True
40 block5_sepconv2 True
41 block5_sepconv2_bn True
42 b

In [50]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(ft_model.get_layer('xception').layers[:90]):
  layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)
ft_model.summary()

0 input_3 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_4 False
13 block2_pool False
14 batch_normalization_4 False
15 add_12 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_5 False
23 block3_pool False
24 batch_normalization_5 False
25 add_13 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_6 False
33 block4_pool False
34 batch_normalization_6 False
35 add_14 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepco

In [51]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [52]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = aug_train_gen,
    batch_size = 32,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

Epoch 1/200
84/84 [==============================] - 18s 172ms/step - loss: 1.6032 - accuracy: 0.3905 - val_loss: 1.7802 - val_accuracy: 0.4167
Epoch 2/200
84/84 [==============================] - 15s 173ms/step - loss: 1.4439 - accuracy: 0.4691 - val_loss: 1.3835 - val_accuracy: 0.5000
Epoch 3/200
84/84 [==============================] - 13s 160ms/step - loss: 1.3730 - accuracy: 0.5002 - val_loss: 1.3501 - val_accuracy: 0.5149
Epoch 4/200
84/84 [==============================] - 13s 159ms/step - loss: 1.3328 - accuracy: 0.4983 - val_loss: 1.2013 - val_accuracy: 0.5804
Epoch 5/200
84/84 [==============================] - 13s 158ms/step - loss: 1.2751 - accuracy: 0.5485 - val_loss: 1.6517 - val_accuracy: 0.4107
Epoch 6/200
84/84 [==============================] - 14s 170ms/step - loss: 1.2416 - accuracy: 0.5563 - val_loss: 1.4387 - val_accuracy: 0.4851
Epoch 7/200
84/84 [==============================] - 14s 171ms/step - loss: 1.2450 - accuracy: 0.5391 - val_loss: 1.5370 - val_accuracy:

In [53]:
ft_model.save('FineTuningModel')
del ft_model

In [54]:
model_aug = tfk.models.load_model("FineTuningModel")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with VGG")
print(model_aug_test_metrics)

17/17 [==============================] - 2s 66ms/step - loss: 1.0426 - accuracy: 0.6505

Test metrics with VGG
{'loss': 1.042563796043396, 'accuracy': 0.6504672765731812}
